In [1]:
import numpy as np

In [2]:
def g(X, val, op):
    if op == 1:
        # op = 1, sign(X > val)
        return np.sign(X - val)
    else:
        # op = -1, sign(X < val)
        return np.sign(val - X)

m = 10
n = 1

X = np.arange(m).reshape((m, n))
y = np.array([1., 1., 1., -1., -1., -1., 1., 1., 1., -1.])
w = np.ones(m) / m
all_sps = (X[:-1, 0] + X[1:, 0]) / 2
all_ops = np.ones(m - 1)
all_ers = np.ones(m - 1)

steps = 100
er_threhold = 1e-10
epsilon = 1e-16

# calculate all errors
for i, s in enumerate(all_sps):
    all_ers[i] = np.sum(w * (g(X.ravel(), s, all_ops[i]) != y))
    if all_ers[i] > 0.5:
        all_ops[i] = - all_ops[i]
        all_ers[i] = 1. - all_ers[i]

# choose 'm_weak_classifiers' of weak classifier
m_weak_classifiers = m - 1
a = np.zeros(m_weak_classifiers)
rkn_idx = np.argsort(all_ers)[:m_weak_classifiers]
sps = all_sps[rkn_idx]
ops = all_ops[rkn_idx]
ers = all_ers[rkn_idx]

# show all weak classifiers
print('- All the optional weak classifiers as below:')
for i in range(m_weak_classifiers):
    op = ''
    if ops[i] == 1:
        op = '>'
    else:
        op = '<'
    print('G%d(x) = sign(x%s%.1f), er = %.4f' % (i, op, sps[i], ers[i]))

# run N epoches, if err < 1e-10, break
print('- Steps run:')
for step in range(steps):
    # choose best weak classifier of current epoch
    ers = np.ones(m_weak_classifiers)
    for i, s in enumerate(sps):
        ers[i] = np.sum(w * (g(X[:, 0], s, ops[i]) != y))
    idx = np.argsort(ers)[0]
    
    # calculate parameters iteratively
    a[idx] = 1 / 2 * np.log((1 - ers[idx]) / np.maximum(ers[idx], epsilon))
    z = np.sum(w * np.exp(- a[idx] * y * g(X.ravel(), sps[idx], ops[idx])))
    w = w / z * np.exp(- a[idx] * y * g(X.ravel(), sps[idx], ops[idx]))
        
    # estimate
    res = 0.
    for i in range(m_weak_classifiers):
        res += a[i] * g(X.ravel(), sps[i], ops[i])
    er = np.sum(np.sign(res) != y)
    
    print('[Step %d] G%d(x), a[%d]=%.4f, z=%.4f, w=%s, er=%.4f' % (step, idx, idx, a[idx], z, w, er/m))
    
    if er < er_threhold:
        # show function
        func_idxs = []
        for i in range(m_weak_classifiers):
            if a[i] != 0.:
                func_idxs.append(i)
        
        func = 'G(x) = sign['
        for i in func_idxs:
            func = '%s%.4f*G%d(x)' % (func, a[i], i)
            if i < len(func_idxs):
                func = '%s + ' % (func)
                
        func = '%s]' % (func)
        print('- Final G(x):')
        print(func)
        break

- All the optional weak classifiers as below:
G0(x) = sign(x<2.5), er = 0.3000
G1(x) = sign(x<8.5), er = 0.3000
G2(x) = sign(x<1.5), er = 0.4000
G3(x) = sign(x<3.5), er = 0.4000
G4(x) = sign(x>5.5), er = 0.4000
G5(x) = sign(x<7.5), er = 0.4000
G6(x) = sign(x>0.5), er = 0.5000
G7(x) = sign(x>4.5), er = 0.5000
G8(x) = sign(x>6.5), er = 0.5000
- Steps run:


NameError: name 'epoch' is not defined